In [ ]:
def inference(embedding):
    """Build the Skip-CNN model.
    
  Args:    embedding
  Returns:    cnn_output
    
  """
    
    #convolution layer
    
    filter_sizes = [3, 4, 5]
    num_filters = 128

    pooled_outputs = []
    for i, filter_size in enumerate(filter_sizes):
        with tf.name_scope("conv-maxpool-%s" % filter_size):
            # Convolution Layer
            filter_shape = [filter_size, embedding_size, 1, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(
                embedded_chars_expanded,
                W,
                strides=[1, 1, 1, 1],
                padding="VALID",
                name="conv")
            # Apply nonlinearity
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            # Max-pooling over the outputs
            pooled = tf.nn.max_pool(
                h,
                ksize=[1, sequence_length - filter_size + 1, 1, 1],
                strides=[1, 1, 1, 1],
                padding='VALID',
                name="pool")
            pooled_outputs.append(pooled)

    num_filters_total = num_filters * len(filter_sizes)
    h_pool = tf.concat( pooled_outputs, 3)
    h_pool_flat = tf.reshape(h_pool, [-1, num_filters_total])

    return h_pool_flat

    

In [ ]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops

def cos_s(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(384):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

def loss(cnn_output):
    """Side cosine_similarity - Center cosine_similarity
  
  Args:
    logits: cnn_output ( sentence_number * 384 : 3 kernel * 128 kernel number )
    
  Returns:
    Loss tensor of type float.
  """
    unstack_cnn_output = tf.unstack(cnn_output, axis=0)
    unstack_fin = session.run(unstack_cnn_output)
    
    newinput=[]
    cos_s_sum = 0
    
    num_sentense = len(unstack_fin)
    
    for si in range(num_sentense - 3):
        for ssi in range(si, num_sentense - 2):
            if not si == ssi:
                calc = (cos_s(unstack_fin[si], unstack_fin[ssi]) + cos_s(unstack_fin[si+2], unstack_fin[ssi+2]))/2 - cos_s(unstack_fin[si+1], unstack_fin[ssi+1])
                newinput.append(calc)
                cos_s_sum += calc
    
    return tf.convert_to_tensor(cos_s_sum)
    

In [ ]:
logits = inference(embedding)
loss = loss(cnn_output)
train_op = train(loss, global_step)